In [9]:
#+++ версия 2020.09.21 инициализация посредством global init()
from importlib import reload
import adds
adds = reload(adds)
from adds import * # import adds functions

init()

ii(dfr,1)
# read_rmsp()

[  0.00   ₀.₀]15:21:33 ₂₀₂₁₋₀₃₋₂₄  init()


,1,2,3,4,5,6,7,8,9,10
0,HTML,__builtins__,__cached__,__doc__,__file__,__loader__,__name__,__package__,__spec__,_log
1,_o,adds,datetime,df_info,dfr,disp,display,glob,gmtime,grpp
2,ii,init,isfile,join,l,listdir,math,merge_OKVED,np,os
3,pd,read_rmsp,reload,requests,shutil,sizeof_fmt,start,strftime,time,tm
4,tmpxls,xl_rowcol_to_cell,zipfile,-,-,-,-,-,-,-


,name,rows,cols,memory,keys
0,'jql',451,7,"25,384","['key','Created','Resolved','fields.summary','fields.description','Report type','Report Project']"


In [38]:
# 2021-03-23 1.0 Загрузка из Jira

from atlassian import Jira

jira_instance = Jira(
    url = "https://jira.hyundai.ru/",
    username = "S.Savinskiy",
    password = "Solid@2020",
)

tm()
# jql = 'project in ("GM", "HM") and "Issue Type Report" is not EMPTY \
def get_from_jira(start,end):

    jql = 'created >= '+start+' AND created <= '+end+' \
            AND"Project Report" is not EMPTY \
            and "Issue Type Report" is not EMPTY \
            and resolutiondate is not EMPTY \
            and created is not EMPTY'

    results = jira_instance.jql( jql  ,
                                limit = 3000, fields=[
                              "issuetype",
                              "created",
                              "resolutiondate",
                              "description",
                              "jira_instance", 
                              "customfield_10401",
                              "customfield_10402",
                              "status", "summary"
                             ]
                                )
    # Load the results into a DataFrame. The list of issues is under the "issues" key of the results object.

    dfr['jql'] = df = pd.json_normalize(results["issues"])

    tm('2. read jql')


    df['Report type'] = [x[0] for x in df['fields.customfield_10401'] ]
    df['Report Project'] = [ x[0]['value'] if type(x) == list  else x for x in df['fields.customfield_10402']  ]
    
    df['Created'] = df['fields.created'].apply(lambda x: 
                                        datetime.datetime.strptime(x,'%Y-%m-%dT%H:%M:%S.%f%z').replace(tzinfo=None))

    df['Resolved'] = df['fields.resolutiondate'].apply(lambda x: 
                                        datetime.datetime.strptime(x,'%Y-%m-%dT%H:%M:%S.%f%z').replace(tzinfo=None))

    df['Created'] = df['Created'].dt.tz_localize(None)
    df['Resolved'] = df['Resolved'].dt.tz_localize(None)

    # Define which fields we care about using dot notation for nested fields.
    
    FIELDS_OF_INTEREST = ["key", 
                          "Created", 
                          "Resolved", 
    #                       "fields.issuetype.name", 
    #                       "fields.status.name",
                          "fields.summary", 
                          "fields.description",
                          "Report type",
    #                       "customfield_10401",  # Issue Type Report
                          "Report Project"]
    
    # Filter to only display the fields we care about. To actually filter them out use df = df[FIELDS_OF_INTEREST].
    
    dfr['jql'] = df = df[FIELDS_OF_INTEREST]
    
    return df
    # df = df[df['fields.key'] in ['HM-54']].sample(5).T

    
df = get_from_jira('2020-01-01','2020-06-01')    
print (df.count())

df = df.append(get_from_jira('2020-06-01','2020-12-31')) 
print (df.count())

dfr['jql'] = df
 

tmpxls(df,'jira')
ii(dfr,1)

tm('************ end')
# df.sample(10).T

*** Start at: 16:07:23 ₂₀₂₁₋₀₃₋₂₄  ************************************************************
[  0.39   ₀.₄]16:07:23 ₂₀₂₁₋₀₃₋₂₄  2. read jql
key                   345
Created               345
Resolved              345
fields.summary        345
fields.description    292
Report type           345
Report Project        345
dtype: int64
[  0.67   ₁.₁]16:07:24 ₂₀₂₁₋₀₃₋₂₄  2. read jql
key                   1091
Created               1091
Resolved              1091
fields.summary        1091
fields.description     962
Report type           1091
Report Project        1091
dtype: int64
[WinError 183] Невозможно создать файл, так как он уже существует: 'tmp/'


C:\Users\Sergey.S\anaconda3\lib\site-packages\xlsxwriter\workbook.py:329: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


,name,rows,cols,memory,keys
0,'jql',1091,7,"69,824","['key','Created','Resolved','fields.summary','fields.description','Report type','Report Project']"


[  0.40   ₁.₅]16:07:24 ₂₀₂₁₋₀₃₋₂₄  ************ end


In [145]:
# 2021-03-17 v1.1  JIRA API report 
# 2021-03-17 v1.2 Добавлено считывание всех файлов из рабочей папки
#            v1.3 Добавлена проверка на остутствие типа ISSUE 
#                 [x for x in df['Пользовательское поле (Issue Type Report)'].unique() if str(x) != 'nan']

import numpy as np
import pandas as pd
import os
   
# удаляем все повторы если одни и теже issues дублируются  в разных выборках    
df = dfr['jql']

df = df.drop_duplicates() 

print (df.count())  


# смотрим сколько значений в выгрузе заполнено
print (df.count())  


['key','Created','Resolved','fields.summary','fields.description','Report type','Report Project']


date_report = 'Resolved' # или Cоздано

df['По месяцам'] = pd.to_datetime(df[date_report], errors='coerce').dt.strftime('%Y-%m')


FMT = '%d.%m.%Y %H:%M'

df['delta'] = (df['Resolved'] - df['Created']).dt.total_seconds()/60/60 #расчет времени работ в часах



# поля для группировки
g_fields = [
            'Пользовательское поле (Issue Type Report)',
            'Пользовательское поле (Project Report)',
            'Тема', 
            'Описание',
           ]




g_fields = [ 
             'Report type',
             'Report Project',
             'fields.summary',
             'fields.description',
           ]


excnt = 1

global dfgg
dfgg = {}

for m in [x for x in df['По месяцам'].unique() if str(x) != 'nan']:
    res_dir = fpath+"_по полю "+date_report+"/"+ m + "/"
    if not os.path.exists(res_dir):
        os.makedirs(res_dir)
    
    for issue_type in [x for x in df['Report type'].unique() if str(x) != 'nan']:

#         Отбираем 
        
        flt = (df['По месяцам'] == m) & (df['Report type'] == issue_type)
        dff = df[ flt ] 
        
        dfg = dff.groupby(g_fields)['delta'].sum()
        
        fname = res_dir+ m+'_'+issue_type+'_report_group_by_month.xlsx'
        
        dfgg[m+'_'+issue_type] = dfg.reset_index()
        
       
        try : 
            dfg.to_excel(fname)
#             print (fname)
#             to_word(dfg,fname)
            
        except Exception as e:
            excnt = excnt + 1
            print(str(excnt) + ". err:", m, issue_type, ":" , e  )
        
    
# disp(dfg.reset_index())

key                   1091
Created               1091
Resolved              1091
fields.summary        1091
fields.description     962
Report type           1091
Report Project        1091
dtype: int64
key                   1091
Created               1091
Resolved              1091
fields.summary        1091
fields.description     962
Report type           1091
Report Project        1091
dtype: int64
2. err: 2020-08 разработка : index 0 is out of bounds for axis 0 with size 0
3. err: 2020-07 разработка : index 0 is out of bounds for axis 0 with size 0
4. err: 2020-10 разработка : index 0 is out of bounds for axis 0 with size 0
5. err: 2020-09 разработка : index 0 is out of bounds for axis 0 with size 0
6. err: 2021-02 разработка : index 0 is out of bounds for axis 0 with size 0
7. err: 2021-02 техподдержка : index 0 is out of bounds for axis 0 with size 0
8. err: 2020-03 разработка : index 0 is out of bounds for axis 0 with size 0
9. err: 2020-03 доработка : [Errno 13] Permission denie

In [64]:
ii(dfgg,1)

,name,rows,cols,memory,keys
0,'2020-06_разработка',33,5,"1,448","['Report type','Report Project','fields.summary','fields.description','delta']"
1,'2020-06_доработка',26,5,"1,168","['Report type','Report Project','fields.summary','fields.description','delta']"
2,'2020-06_техподдержка',34,5,"1,488","['Report type','Report Project','fields.summary','fields.description','delta']"
3,'2020-08_разработка',0,5,124,"['Report type','Report Project','fields.summary','fields.description','delta']"
4,'2020-08_доработка',95,5,"3,928","['Report type','Report Project','fields.summary','fields.description','delta']"
5,'2020-08_техподдержка',78,5,"3,248","['Report type','Report Project','fields.summary','fields.description','delta']"
6,'2020-05_разработка',1,5,168,"['Report type','Report Project','fields.summary','fields.description','delta']"
7,'2020-05_доработка',7,5,408,"['Report type','Report Project','fields.summary','fields.description','delta']"
8,'2020-05_техподдержка',11,5,568,"['Report type','Report Project','fields.summary','fields.description','delta']"
9,'2020-07_разработка',0,5,124,"['Report type','Report Project','fields.summary','fields.description','delta']"


In [253]:
match = re.search(r'{(.+?)}','sAsdasdasS {report str} dasd')
print ( match[1] if match else "-")

report str


In [267]:
from docx import Document
import docx
import re


fres = r'C:\\!HML\00.General\02.Отчеты по разработке\\'

fname = r'Jira Report.docx'


def docx_report (df, fres = 'res//~default.docx',  data = {"report_date":"31.02.2020г."}):
    
    from docx.enum.text import WD_ALIGN_PARAGRAPH
    
    fpath = r'C:\\!HML\00.General\02.Отчеты по разработке\\'
    
    doc = Document(fpath + '_Отчет шаблон.docx')
    doc.tables #a list of all tables in document

    
    
    df['fields.description2'] = df['fields.description'].apply(lambda x:  re.sub(r'[\r\n]+', '\n', x).strip())    
    
    cols = ['Report Project','fields.summary','fields.description2','delta']
  
    
# 2021-03-26 Заменяем плейсхолдеры    
    
    for paragraph in doc.paragraphs:
        ptxt = paragraph.text
        
#         print ("s"+ptxt)
        
        place_holder_name = re.search(r'{(.+?)}',ptxt)
        
        match = re.search(r'{(.+?)}', ptxt)
        place_holder_name = match[1] if match else "-"

        print (place_holder_name)
        
        ptxt = ptxt.replace("{"+place_holder_name+"}" , data[place_holder_name] ) if place_holder_name in data else ptxt

        print ("e:" + ptxt)
        
        paragraph.text = ptxt

   
    l=1
     
    
    for i,r in df[cols][:60].iterrows():
#         print (r);
        c = 0
        for idx,val in r.iteritems():
               
            doc.tables[0].cell(l, c).text = str(val) #r[0] +' '+ str(val)
            font = doc.tables[0].cell(l, c).paragraphs[0].runs[0].font
            font.size = docx.shared.Pt(9)
            c = c +1
#             doc.tables[0].cell(l, c).text = str(1000)
        doc.tables[0].add_row() #ADD ROW HERE
       
        l = l + 1
   
    print (fres)
    
    sr = 1
    for rp in df.groupby(['Report Project']).count().reset_index()['Report type'].to_list():
        print(sr,rp)
        project_name = doc.tables[0].cell(sr, 0).text
        
        doc.tables[0].cell(sr, 0).merge(doc.tables[0].cell(sr+rp-1, 0))
        doc.tables[0].cell(sr, 3).merge(doc.tables[0].cell(sr+rp-1, 3))

        doc.tables[0].cell(sr, 0).text = project_name
        sr = sr + rp
    

    doc.save(fres) 
#     f'{sourcesystem}_{batchid}_{extractname}_{loadtype}'


df = dfgg['2020-12_техподдержка'] #.reset_index()
df = dfgg['2020-04_доработка'] #.reset_index()
df = dfgg['2020-11_техподдержка'] #.reset_index() 16

# display(HTML("<a href ='C://!HML/00.General/02.Отчеты по разработке/docx/20-12_техподдержка.docx'>link</a>"))


data = {"report_date":"31.02.2020г.",
        "agreement_subject":"технической поддержке и доработке ПО «Hyundai Mobility»",
        "agreement_name":"Договор на оказание услуг №2 от 11.10.2019г.",
        "report_month": "01.01-31.01.2020г"
       }


fres = r'C:\\!HML\00.General\02.Отчеты по разработке\docx\20-12_техподдержка.docx'

docx_report (df, fres = fres, data = data)


disp(df[:3])

-
e:Отчет №3 об оказанных услугах по технической поддержке и доработке  ПО «Hyundai Mobility»
agreement_subject
e:технической поддержке и доработке ПО «Hyundai Mobility»
report_date
e:Дата: 31.02.2020г. 31.01.2020г.
-
e:Заказчик: ООО Хендэ Мотор СНГ
agreement_name
e:Основание: Договор на оказание услуг №2 от 11.10.2019г. Договор на оказание услуг №2 от 11.10.2019г.
report_month
e:Период оказания услуг: 01.01-31.01.2020г 01.01-31.01.2020г.
-
e:Доработка ПО “Hyundai Mobility” согласно ТЗ №4 от _________ 2020г.
-
e:
-
e:
-
e:Генеральный директор ООО «Хендэ Мобилити Лаб» 			Калицев А.В.
-
e:Генеральный директор ООО «Хендэ Мотор СНГ» 	             		 Хван К.Й.
C:\\!HML\00.General\02.Отчеты по разработке\docx\20-12_техподдержка.docx
1 4
5 10
15 2


PermissionError: [Errno 13] Permission denied: 'C:\\\\!HML\\00.General\\02.Отчеты по разработке\\docx\\20-12_техподдержка.docx'

In [276]:
# 2021-03-26 подготовка отчетов по календарю

fpath = r'C:\!HML\00.General\02.Отчеты по разработке\\'

dfr['calendar'] = df =  pd.read_excel(fpath + r'Календарь продуктов Лаба.xlsx')

df

,Продукт,Номер договора/доп.соглашения,Дата договора/доп.соглашения,Значение,Дата акта
0,ПО Software TMU,Спецификация 2 к С 2020-247,2020-09-01 00:00:00,1,2020-11-30 00:00:00
1,Приложение Genesis Mobility,Спецификация 1 к С 2020-447,2020-06-26 00:00:00,2,2020-07-01 00:00:00
2,Приложение Mobikey,Спецификация 1 к С 2020-447,2020-06-26 00:00:00,2,2020-07-01 00:00:00
3,Техническая поддержка ПО Genesis Mobility,ДС 1 к договору 2,2020-07-01 00:00:00,4,Последний день квартала
4,Техническая поддержка ПО Mobikey,ДС 1 к договору 3,2020-07-01 00:00:00,4,Последний день квартала
5,Доработка ПО Genesis Mobility,ДС 1 к договору 4,2020-07-01 00:00:00,4,NaN
6,Доработка ПО Mobikey,ДС 1 к договору 5,2020-07-01 00:00:00,4,NaN
7,Приложение «Hyundai Mobility»,1,2019-09-01 00:00:00,3,10 октября 2019 года
8,CRM Mobility,1,2019-09-01 00:00:00,3,Часть Hyundai Mobility
9,CRM Техник,1,2019-09-01 00:00:00,3,Часть Hyundai Mobility


In [280]:
df = dfr['jql']
df.groupby(['Report Project'])['Report Project'].count()

Report Project
CRM Mobility                        551
Доработка ПО Genesis Mobility        75
Доработка ПО Hyundai Mobility       223
Доработка ПО Mobikey                 44
ПО Software TMU                       4
Приложение Genesis Mobility           9
Приложение Mobikey                   31
Техподдержка ПО Genesis Mobility     25
Техподдержка ПО Hyundai Mobility    118
Техподдержка ПО Mobikey              11
Name: Report Project, dtype: int64

In [281]:
ii(dfr,1)

,name,rows,cols,memory,keys
0,'jql',1091,7,"69,824","['key','Created','Resolved','fields.summary','fields.description','Report type','Report Project']"
1,'calendar',15,5,728,"['Продукт','Номер договора/доп.соглашения','Дата договора/доп.соглашения','Значение','Дата акта']"
